<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_stock_trading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog uses FinRL to reproduce the paper: Practical Deep Reinforcement Learning Approach for Stock Trading, Workshop on Challenges and Opportunities for AI in Financial Services, NeurIPS 2018.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
!pip install yfinance
!pip install stockstats
!pip install pyfolio
!pip install gym

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

d:\installedsoftware\anaconda3\envs\py36-notebook\lib\site-packages\pyfolio\pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [3]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [4]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [5]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [6]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-01-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
df.shape

(90630, 8)

In [8]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.787006,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.698216,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.592946,40980600,CSCO,4


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [9]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [10]:
processed.shape
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,close_120_sma,close_30_ema,close_60_ema,close_120_ema,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.787006,746015200,AAPL,4,0.0,3.012162,...,100.0,66.666667,100.0,2.787006,2.787006,2.787006,2.787006,2.787006,2.787006,0.0
1,2009-01-02,18.570000,19.520000,18.400000,15.698216,10955700,AXP,4,0.0,3.012162,...,100.0,66.666667,100.0,15.698216,15.698216,15.698216,15.698216,15.698216,15.698216,0.0
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.0,3.012162,...,100.0,66.666667,100.0,33.941101,33.941101,33.941101,33.941101,33.941101,33.941101,0.0
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4,0.0,3.012162,...,100.0,66.666667,100.0,32.830360,32.830360,32.830360,32.830360,32.830360,32.830360,0.0
4,2009-01-02,16.410000,17.000000,16.250000,12.592946,40980600,CSCO,4,0.0,3.012162,...,100.0,66.666667,100.0,12.592946,12.592946,12.592946,12.592946,12.592946,12.592946,0.0


In [11]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

In [12]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,close_120_sma,close_30_ema,close_60_ema,close_120_ema,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.787006,746015200.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,2.787006,2.787006,2.787006,2.787006,2.787006,2.787006,0.0
1,2009-01-02,AXP,18.570000,19.520000,18.400000,15.698216,10955700.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,15.698216,15.698216,15.698216,15.698216,15.698216,15.698216,0.0
2,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,33.941101,33.941101,33.941101,33.941101,33.941101,33.941101,0.0
3,2009-01-02,CAT,44.910000,46.980000,44.709999,32.830360,7117200.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,32.830360,32.830360,32.830360,32.830360,32.830360,32.830360,0.0
4,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.592946,40980600.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,12.592946,12.592946,12.592946,12.592946,12.592946,12.592946,0.0
5,2009-01-02,CVX,74.230003,77.300003,73.580002,47.373642,13695900.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,47.373642,47.373642,47.373642,47.373642,47.373642,47.373642,0.0
6,2009-01-02,DD,21.605234,22.060680,20.993229,15.336650,13251037.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,15.336650,15.336650,15.336650,15.336650,15.336650,15.336650,0.0
7,2009-01-02,DIS,22.760000,24.030001,22.500000,20.597496,9796600.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,20.597496,20.597496,20.597496,20.597496,20.597496,20.597496,0.0
8,2009-01-02,GS,84.019997,87.620003,82.190002,72.559456,14088500.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,72.559456,72.559456,72.559456,72.559456,72.559456,72.559456,0.0
9,2009-01-02,HD,23.070000,24.190001,22.959999,17.797922,14902500.0,4.0,0.0,3.012162,...,100.0,66.666667,100.0,17.797922,17.797922,17.797922,17.797922,17.797922,17.797922,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2018-12-31
## Trade data split: 2019-01-01 to 2020-09-30

In [13]:
train = data_split(processed_full, '2009-01-01','2019-01-01')
trade = data_split(processed_full, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [14]:
initialData = train.loc[0,:]
print(initialData)

         date   tic       open       high        low      close       volume  \
0  2009-01-02  AAPL   3.067143   3.251429   3.041429   2.787006  746015200.0   
0  2009-01-02   AXP  18.570000  19.520000  18.400000  15.698216   10955700.0   
0  2009-01-02    BA  42.799999  45.560001  42.779999  33.941101    7010200.0   
0  2009-01-02   CAT  44.910000  46.980000  44.709999  32.830360    7117200.0   
0  2009-01-02  CSCO  16.410000  17.000000  16.250000  12.592946   40980600.0   
0  2009-01-02   CVX  74.230003  77.300003  73.580002  47.373642   13695900.0   
0  2009-01-02    DD  21.605234  22.060680  20.993229  15.336650   13251037.0   
0  2009-01-02   DIS  22.760000  24.030001  22.500000  20.597496    9796600.0   
0  2009-01-02    GS  84.019997  87.620003  82.190002  72.559456   14088500.0   
0  2009-01-02    HD  23.070000  24.190001  22.959999  17.797922   14902500.0   
0  2009-01-02   IBM  83.889999  87.589996  83.889999  59.180256    7558200.0   
0  2009-01-02  INTC  14.690000  15.25000

In [15]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,close_120_sma,close_30_ema,close_60_ema,close_120_ema,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.439735,148158800.0,2.0,-2.013469,44.430054,...,37.867359,-91.575723,42.250808,41.155815,46.409359,48.900092,41.352494,44.458658,46.304234,51.409187
0,2019-01-02,AXP,93.910004,96.269997,93.769997,92.042946,4175400.0,2.0,-3.403805,109.907410,...,41.204998,-97.759092,26.709417,99.719922,100.286694,100.394721,97.239951,99.212594,99.167165,51.409187
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-5.550592,339.116375,...,47.010000,-21.712382,13.611972,314.427160,331.956148,337.512527,316.591202,325.615852,330.278999,51.409187
0,2019-01-02,CAT,124.029999,127.879997,123.000000,118.137177,4783200.0,2.0,-0.680049,125.145761,...,48.229091,-5.147002,0.873482,117.640500,119.834356,126.242992,117.688802,120.504942,124.644566,51.409187
0,2019-01-02,CSCO,42.279999,43.200001,42.209999,39.772106,23833500.0,2.0,-0.945558,46.120801,...,44.872560,-87.538213,29.529377,41.863034,42.137596,41.998078,41.287111,41.840720,41.620403,51.409187


In [16]:
config.TECHNICAL_INDICATORS_LIST

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'close_120_sma',
 'close_30_ema',
 'close_60_ema',
 'close_120_ema']

In [17]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 421


In [18]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

## Environment for Training



In [19]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [20]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [25]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [26]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=1000)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

### Model 2: DDPG

In [27]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [28]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=1000)

[1000000, 16.090686798095703, 50.84724807739258, 63.82965850830078, 68.9310302734375, 15.391002655029297, 78.00062561035156, 36.09136199951172, 45.454647064208984, 112.89419555664062, 50.98896408081055, 140.82015991210938, 16.230836868286133, 55.96622848510742, 34.79741287231445, 28.365230560302734, 71.26537322998047, 74.27186584472656, 32.05595016479492, 22.98516845703125, 23.8995361328125, 17.483909606933594, 53.18644332885742, 42.709110260009766, 60.0010986328125, 48.292266845703125, 35.528656005859375, 29.847227096557617, 29.557008743286133, 56.27716064453125, 61.08891677856445, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.4790281612272622, 0.352971743064856, 0.7102643955258543, 1.0782265988359399, 0.40539747734638176, 0.5972740143129016, 0.9219816875346467, 0.34665957292345695, 2.1366686254966396, -0.1299040345792477, 0.37170635738817737, 0.06575029680240618, 0.28206866616658743, 0.6303649480151137, 0.0011612970931587085, 0.761338465

### Model 3: PPO

In [29]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [30]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=1000)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

### Model 4: TD3

In [39]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [40]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=1000)

[1000000, 32.50059127807617, 74.83963775634766, 163.9823455810547, 87.8850326538086, 29.833024978637695, 92.54185485839844, 82.32376861572266, 105.45314025878906, 233.69862365722656, 131.2222900390625, 148.79367065429688, 32.40830993652344, 106.72248840332031, 80.83092498779297, 36.15778732299805, 115.09774780273438, 164.869384765625, 57.48778533935547, 60.56845474243164, 55.59139633178711, 27.138832092285156, 80.93218994140625, 64.38614654541016, 110.1478500366211, 150.4595947265625, 85.43305206298828, 41.12411117553711, 76.33426666259766, 65.29778289794922, 64.70258331298828, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.079358096913154, 0.9969893240458418, 3.7708223412194286, 1.041743360226974, 0.7820935349617777, -0.6173242681077653, 1.2651806649912345, 0.7687065547034564, 3.8854876907754488, 1.950526936667245, 2.6415959580452864, -0.059886542566353285, 1.111709737361167, 1.3723485776938844, -0.051027585668855124, 1.381668797651102, 1.9

### Model 5: SAC

In [41]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [42]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=1000)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [43]:
data_turbulence = processed_full[(processed_full.date<'2019-01-01') & (processed_full.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [44]:
insample_turbulence.turbulence.describe()

count    2516.000000
mean       33.278196
std        33.999877
min         0.000000
25%        15.233733
50%        25.167097
75%        39.290279
max       332.849213
Name: turbulence, dtype: float64

In [45]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [46]:
turbulence_threshold

332.8492127964834

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [47]:
trade = data_split(processed_full, '2019-01-01','2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 380, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

[1000000, 38.439735412597656, 92.0429458618164, 314.6451416015625, 118.13717651367188, 39.7721061706543, 99.26136779785156, 74.09832763671875, 107.65434265136719, 163.94699096679688, 162.94903564453125, 101.94241333007812, 44.27648162841797, 120.20957946777344, 91.72384643554688, 43.620418548583984, 166.7007598876953, 176.6388397216797, 70.6997299194336, 98.37548065185547, 72.4448013305664, 37.23637771606445, 85.57616424560547, 64.40036010742188, 111.04877471923828, 234.7300567626953, 130.8709716796875, 50.405242919921875, 62.57354736328125, 89.26032257080078, 59.68837356567383, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.01346910769135, -3.403804612767189, -5.5505923876932, -0.6800493576991471, -0.9455581149819992, -1.91175117903758, -1.2729275169320005, -1.651912931269365, -8.096245348729553, -1.7997391293691862, -2.2648934134193013, -0.2572228966760548, -4.013110074480451, -2.5660852949502413, -0.4048977948138841, -1.5590910367507433,

In [48]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,close_120_sma,close_30_ema,close_60_ema,close_120_ema,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.439735,148158800.0,2.0,-2.013469,44.430054,...,37.867359,-91.575723,42.250808,41.155815,46.409359,48.900092,41.352494,44.458658,46.304234,51.409187
0,2019-01-02,AXP,93.910004,96.269997,93.769997,92.042946,4175400.0,2.0,-3.403805,109.907410,...,41.204998,-97.759092,26.709417,99.719922,100.286694,100.394721,97.239951,99.212594,99.167165,51.409187
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-5.550592,339.116375,...,47.010000,-21.712382,13.611972,314.427160,331.956148,337.512527,316.591202,325.615852,330.278999,51.409187
0,2019-01-02,CAT,124.029999,127.879997,123.000000,118.137177,4783200.0,2.0,-0.680049,125.145761,...,48.229091,-5.147002,0.873482,117.640500,119.834356,126.242992,117.688802,120.504942,124.644566,51.409187
0,2019-01-02,CSCO,42.279999,43.200001,42.209999,39.772106,23833500.0,2.0,-0.945558,46.120801,...,44.872560,-87.538213,29.529377,41.863034,42.137596,41.998078,41.287111,41.840720,41.620403,51.409187


In [49]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

[1000000, 38.439735412597656, 92.0429458618164, 314.6451416015625, 118.13717651367188, 39.7721061706543, 99.26136779785156, 74.09832763671875, 107.65434265136719, 163.94699096679688, 162.94903564453125, 101.94241333007812, 44.27648162841797, 120.20957946777344, 91.72384643554688, 43.620418548583984, 166.7007598876953, 176.6388397216797, 70.6997299194336, 98.37548065185547, 72.4448013305664, 37.23637771606445, 85.57616424560547, 64.40036010742188, 111.04877471923828, 234.7300567626953, 130.8709716796875, 50.405242919921875, 62.57354736328125, 89.26032257080078, 59.68837356567383, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.01346910769135, -3.403804612767189, -5.5505923876932, -0.6800493576991471, -0.9455581149819992, -1.91175117903758, -1.2729275169320005, -1.651912931269365, -8.096245348729553, -1.7997391293691862, -2.2648934134193013, -0.2572228966760548, -4.013110074480451, -2.5660852949502413, -0.4048977948138841, -1.5590910367507433,

[1000000, 132.26734924316406, 120.12644958496094, 214.05999755859375, 180.24192810058594, 44.07402801513672, 83.27289581298828, 70.80415344238281, 181.17999267578125, 262.6781921386719, 263.96588134765625, 122.84745788574219, 49.21628952026367, 156.4047088623047, 125.42272186279297, 54.38731384277344, 213.26780700683594, 173.3614044189453, 81.08755493164062, 221.90890502929688, 141.18296813964844, 36.05856704711914, 137.426513671875, 71.07267761230469, 139.6039276123047, 349.43853759765625, 218.07418823242188, 57.509002685546875, 39.50341033935547, 142.9890899658203, 39.9496955871582, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.8535854042010413, 1.053287663230492, 2.7831664269158125, 1.966865798358782, 0.7341816780368688, -0.13418687362631943, 1.8908233196613082, 8.849684867275954, 9.300764588233164, -1.1098969575164688, 0.9907869217878016, 0.028844635174799294, 2.211265218575562, 2.755051524216597, 0.5908858835766395, -0.2915981586465648

In [50]:
df_account_value.shape

(505, 2)

In [51]:
df_account_value.tail()

,date,account_value
500,2020-12-24,1.225527e+06
501,2020-12-28,1.238559e+06
502,2020-12-29,1.237790e+06
503,2020-12-30,1.249383e+06
504,2020-12-31,1.254300e+06


In [52]:
df_actions.head()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,...,PFE,PG,RTX,TRV,UNH,V,VZ,WBA,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,0,0,100,0,100,100,100,0,100,0,...,0,100,0,100,0,0,0,100,100,0
2019-01-03,0,0,100,0,100,100,100,0,100,0,...,0,100,0,100,0,0,0,100,100,0
2019-01-04,0,0,100,0,100,100,100,0,100,0,...,0,100,0,100,0,0,0,100,100,0
2019-01-07,0,0,100,0,100,100,100,0,100,0,...,0,100,0,100,0,0,0,100,100,0
2019-01-08,0,0,100,0,100,100,100,0,100,0,...,0,100,0,100,0,0,0,100,100,0


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [53]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.119704
Cumulative returns     0.254300
Annual volatility      0.191330
Sharpe ratio           0.688216
Calmar ratio           0.514461
Stability              0.036524
Max drawdown          -0.232679
Omega ratio            1.136214
Sortino ratio          0.942729
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.866602
Daily value at risk   -0.023583
dtype: float64


In [54]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = '2019-01-01',
        end = '2021-01-01')

stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (505, 8)
Annual return          0.144674
Cumulative returns     0.310981
Annual volatility      0.274619
Sharpe ratio           0.631418
Calmar ratio           0.390102
Stability              0.116677
Max drawdown          -0.370862
Omega ratio            1.149365
Sortino ratio          0.870084
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860710
Daily value at risk   -0.033911
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [63]:
!pip install git+https://github.com/quantopian/pyfolio
!pip install empyrical

  Cloning https://github.com/quantopian/pyfolio to c:\users\user\appdata\local\temp\pip-req-build-8ghjcsdd


  Running command git clone -q https://github.com/quantopian/pyfolio 'C:\Users\User\AppData\Local\Temp\pip-req-build-8ghjcsdd'


In [64]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (505, 8)


Start date,2019-01-02
End date,2020-12-31
Total months,24
,Backtest
Annual return,12.0%
Cumulative returns,25.4%
Annual volatility,19.1%
Sharpe ratio,0.69
Calmar ratio,0.51
Stability,0.04
Max drawdown,-23.3%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'